# Analysis of Tournament Matches in Super Smash Bros. Melee
Super Smash Bros. Melee is a child's party game released by Nintendo in 2001. It is a fighting game where players were able to choose from various Nintendo characters and stages to see who could be the last one standing on the stage. Due to the game's unique physics engine and mechanics, players were able to take Melee and turn it into one of the most complex and deep competative games in esports. We're still seeing players push the boundaries of how Melee can be played, even though the game itself has been out for 19 years now. Large scale tournaments, where players from beginners to professionals from around the world gathered for a prize pool, happened regularly before covid. Our goal is to analyze data from one of these major tournaments, specifically Pound 2019, which took place April 19-21st in Laurel, Maryland.

Passionate community members have reverse engineered Melee's code to develop modifications for the game. One of these modifications, called Project Slippi, keeps track of data for two players when they play a match. This data includes most common attacks used in certain positions, most common attack used to kill the opponent, etc. We will be using data from 484 matches from Pound 2019 taken with Slippi.

We chose to analyze data from Melee because we are both very passionate about the game and community and we chose Pound 2019 since we were there and competed in the event.

## How Melee is Played

Melee has 29 total stages but only 6 were deemed "fair" enough for tournament play. In addition, tournament matches are one on one with both players having 4 stocks (or lives). The goal is to take your opponents four stocks before they take yours. The way this is done is by knocking your opponent off the stage to a point where they are unable to make it back, or by hitting the so hard they fly off the sides or top of the screen. In Melee, percent is taken when the player is hit. The more percent the player has, the further they will be knocked back from attacks. It's best to think of percent in Melee as the amount of damage the character has taken and not as actual percent. In tournaments, players will play either best 2 out of 3 matches, or best 3 out of 5 matches in order to progress through brackets.

# Average Openings Per Kill between Fox and Falco
In Melee, the concept of "neutral game" is when neither player is at an inherit advantage from one another. This is when both players are looking for a way to hit one another in order to get a punish. "Punish game" is when one player lands that opening and begins to combo their opponent. The game is constantly changing between the neutral game and punish game as the match progresses. We want to se how many times a Fox player has to win the neutral game compared to how many times a Falco player has to win neutral for in order to get a kill on the other player. 

# Shield Pressure

In Melee, it is possible to bring up a bubble like shield around your character. When you block an attack with your shield, your shield becomes smaller, leaving your more vulnerable to getting hit. In shield, each character has a set of options that they can do to punish the opponent for hitting their shield. Certain characters are good at what is called shield pressuring, the act of safely attacking your opponents shield to wittle it down and get a hit. The characters most well known for having good shield pressure are fox and falco, since they are the fastest characters in the game. Here, we will analyze shield data to see which characters do the most damage to their opponents shields.

# Average Kill Percent Each Character / Spread

The goal of building percent on your opponent is to make it easier to take their stock by hitting them off the stage. The amount of percent doesn't matter when a player is offstage, they will still lose the stock once off screen. That being said, each character has a range in percent that they are usually killed at. Different characters have different aspects of them that make them die earlier or live longer than other characters such as fall speed, weight, and recovery options. Here, we want to analyze kill percents for each character.

# Average GALINT

When the player grabs the ledge of the stage, they are given 30 frames (or half a second) of invulnerability. By performing a technique known as a ledge dash, you are able to quickly let go of ledge and get on the stage. The term GALINT stands for grounded action ledge intangibility. We want to see the average GALINT for the characters that most commonly ledgedash.